In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
train

In [82]:
test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
test

# Data interpretation(The dataframe and shape)


In [4]:
# check all columns
train.columns

In [5]:
train.info()

In [6]:
test.shape

In [7]:
train.shape

In [8]:
train.isnull().sum()

In [9]:
train.describe().columns

In [83]:
Test_id = test['Id']

# Feature imputation : Fill null values and drop data with too many nulls

In [10]:
# Drop features with too many null values and ID.
train.drop(['Id', 'Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'], axis =1, inplace = True)
test.drop(['Id', 'Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'], axis =1, inplace = True)
print(train.shape)
print(test.shape)

In [11]:
#store the numeric data for data interpretation
numericFT = train.select_dtypes(exclude = ["object"])
numericFT.drop('MSSubClass', axis = 1)
numericFT.head()

In [12]:
#function that returns the fill in value(mode) of the datas with the same (sameone, sametwo) features as the given index
def findmode(data,feature, sameone, sametwo, index) :
    # check for values from data set
    sameData = data.loc[(data[sameone] == data[sameone][index]) & (data[sametwo] == data[sametwo][index]), feature]
    mode = sameData.mode()
    # check if there is no data with same sameone feature and sametwo feature
    if mode.empty :
      # use the primary sameone feature only
      sameData = data.loc[data[sameone] == data[sameone][index], feature]
      mode = sameData.mode()
    if mode.empty :
      #if it is empty again, use secondary feature
      sameData = data.loc[data[sametwo] == data[sametwo][index], feature]
      mode = sameData.mode()
    #if it is still empty return null
    if mode.empty :
      return mode
    # If there is more than one mode, return the first mode
    mode = mode.iloc[0]
    return mode

In [13]:
train['LotFrontage'].fillna(method = 'bfill', inplace=True)
train['MasVnrType'].fillna('None', inplace=True)
train['MasVnrArea'].fillna(0, inplace=True)
train['Electrical'].fillna(method = 'bfill', inplace=True)

# get the neighborhood and mssubclass of the houses where value is na
# find the mode of the feature from the same neighborhood and mssubclass.
# fill it into the na.
fillinFeatures = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond']
for ft in fillinFeatures :
    train[ft].fillna(findmode(train, ft,  'MSSubClass','Neighborhood', train.index.to_series()), inplace=True)
    # print(train[ft].isnull().sum())

In [14]:
test['LotFrontage'].fillna(method = 'bfill', inplace=True)
test['MasVnrType'].fillna('None', inplace=True)
test['MasVnrArea'].fillna(0, inplace=True)
test['Electrical'].fillna(method = 'bfill', inplace=True)

# get the neighborhood and mssubclass of the houses where value is na
# find the mode of the feature from the same neighborhood and mssubclass.
# fill it into the na.
fillinFeatures = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond']
for ft in fillinFeatures :
    test[ft].fillna(findmode(test, ft,  'MSSubClass','Neighborhood', test.index.to_series()), inplace=True)
    # print(test[ft].isnull().sum())

In [15]:
test.info()

In [16]:
train.info()

In [17]:
train.head()

# feature encoding : turning values into numbers

In [18]:
from sklearn import preprocessing

In [19]:
encodingFt = ['MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType','HouseStyle', 
                'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1','BsmtFinType2', 
              'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition']

In [20]:
for ft in encodingFt :
    print(ft + ':', len(train[ft].unique()))

In [21]:
# Using label_encoder for all strings
label_encoder = preprocessing.LabelEncoder()

for ft in encodingFt :
    train[ft] = label_encoder.fit_transform(train[ft])
    
train.info()

In [22]:
train.head()

In [23]:
# Using label_encoder for all strings
label_encoder = preprocessing.LabelEncoder()

for ft in encodingFt :
    test[ft] = label_encoder.fit_transform(test[ft])

test.info()

In [24]:
catFTCol = encodingFt + ['SalePrice']
categoricFT = train[catFTCol]
categoricFT.head()

# Feature Normalisation and standardization 

In [25]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
scaled_Input_df = ss.fit(train.drop("SalePrice", axis = 1))
train_scaled = ss.transform(train.drop("SalePrice", axis = 1))
test_scaled = ss.transform(test)


In [26]:
print(test_scaled.shape)
train_scaled.shape

In [27]:
train_target = train["SalePrice"].to_numpy()
train_target = train_target.astype('float64')
log_train_target = np.log(train_target)
log_train_target = np.reshape(log_train_target, (-1,1))
#print(log_train_target.shape)
train_scaled = np.append(train_scaled, log_train_target, axis = 1)
print(train_scaled.shape)

In [28]:
scaled_train = pd.DataFrame(train_scaled, columns =  train.columns.values )
scaled_train
scaled_train.head()

In [29]:
scaled_test = pd.DataFrame(test_scaled, columns =  train.drop("SalePrice", axis = 1).columns.values )
scaled_test.head()

# Data interpretation(Correlations and Distributions)

In [30]:
import matplotlib.pyplot as plt
import seaborn as sns

In [31]:
# Plot the distribution of all the numerical data
# fig_1 = scaled_train.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8)

In [32]:
#threshhold to use for both numeric and categorical data
threshhold = 0.3

In [33]:
#correlation for numeric values
corr_matrix = scaled_train.corr()
#replace all features that has a correspondance less than threshhold to 0 for visibility.
corr_matrix[abs(corr_matrix) < threshhold] = 0

#mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
#plt.figure(figsize=(24, 20))
#sns.heatmap(corr_matrix,cmap="Blues", mask=mask, annot = True)
#plt.show()

In [34]:
corr = scaled_train.corr()["SalePrice"][:-1]
drop_list = corr[abs(
    corr) <= threshhold].sort_values(ascending=False)
print(len(drop_list) ,drop_list)

# data cleaning

In [35]:
# since we got the dropping list for numerical and categorical values, we can make this into a list.
droplist = list(drop_list.index)
print(droplist)
scaled_train = scaled_train.drop(droplist, axis = 1)
scaled_test = scaled_test.drop(droplist, axis = 1)

In [36]:
scaled_train.shape
scaled_test.shape

In [37]:
# Plot the distribution of all the numerical data
# fig_3 = scaled_train.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8)

# Prepare data for modelling

In [38]:
from sklearn.model_selection import train_test_split
train_input_scaled, val_input_scaled, train_target_scaled, val_target_scaled = train_test_split(scaled_train.drop('SalePrice', axis = 1).to_numpy(), scaled_train['SalePrice'].to_numpy(),test_size = 0.2, shuffle =True)


In [39]:
train_input_scaled.shape
val_input_scaled.shape
train_target_scaled.shape
val_target_scaled.shape

# Making models

In [40]:
from sklearn.metrics import r2_score,mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.svm import SVR
# from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [41]:
def rsqr_score(test, pred):
    r2 = r2_score(test, pred)
    return r2

def rmse_score(test, pred):
    rmse = np.sqrt(mean_squared_error(test, pred))
    return rmse

In [42]:
models = { 'ridge' : Ridge(),
          'lasso' : Lasso(alpha = 0.001),
          'lr' : LinearRegression(),
          'rfr' : RandomForestRegressor(),
          'xgb': XGBRegressor(),
          'lgb': LGBMRegressor()
    }

In [43]:
for name, model in models.items():
    model.fit(train_input_scaled,train_target_scaled)
    print(name + ' trained')
    # R2 error
    print(name + " score : " + str(model.score(val_input_scaled, val_target_scaled)))
    price_pred = model.predict(val_input_scaled)
    print(name + " rmse error : " + str(rmse_score(val_target_scaled, price_pred)))
    print('--------------------------------------------------------------------------------------------')

Hyperparameter Tuning

In [44]:
from sklearn.model_selection import GridSearchCV

Change Alpha value for ridge

In [45]:
alphas = np.logspace(-5, 5).tolist()
alpha_list = {'alpha': alphas}
ridge_cv = GridSearchCV(Ridge(),alpha_list, cv = 10, n_jobs = -1,verbose = 1)
ridge_cv.fit(train_input_scaled, train_target_scaled)

print(f"Best hyperparameters: {ridge_cv.best_params_}")
print(f"Best R² (train): {ridge_cv.best_score_}")

In [46]:
# Ridge Regressor with the best hyperparameters
ridge_mod = Ridge(alpha=ridge_cv.best_params_["alpha"])
# Fit the model on train set
ridge_mod.fit(train_input_scaled, train_target_scaled)
# Predict on test set
val_prediction = ridge_mod.predict(val_input_scaled).reshape(-1, 1)
# print(val_prediction.shape)
# print(val_input_scaled.shape)
print(f"R²: {rsqr_score(val_target_scaled, val_prediction)}")

Change Alpha value for Lasso

In [47]:
alphas = np.logspace(-5, 5).tolist()
alpha_list = {'alpha': alphas}
lasso_cv = GridSearchCV(Lasso(),alpha_list, cv = 10, n_jobs = -1,verbose = 1)
lasso_cv.fit(train_input_scaled, train_target_scaled)

print(f"Best hyperparameters: {lasso_cv.best_params_}")
print(f"Best R² (train): {lasso_cv.best_score_}")

In [48]:
lasso_mod = Lasso(alpha=lasso_cv.best_params_["alpha"])
# Fit the model on train set
lasso_mod.fit(train_input_scaled, train_target_scaled)
# Predict on test set
val_prediction = lasso_mod.predict(val_input_scaled).reshape(-1, 1)
# print(val_prediction.shape)
# print(val_input_scaled.shape)
print(f"R²: {rsqr_score(val_target_scaled, val_prediction)}")

max_depth, min_sample_split, n_estimators for **random forest regressor**

In [49]:
from sklearn.model_selection import RandomizedSearchCV

In [51]:
max_depth = [10, 20, 30, 50, 80, 100, 150, 200]
max_depth.append(None)
# no limit
min_samples_split = [2, 5, 10, 20, 40, 80]

n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)]

param_list = {'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'n_estimators': n_estimators
             }
rfr_cv = RandomizedSearchCV(RandomForestRegressor(), param_distributions = param_list, 
                        n_iter=10, verbose =1, n_jobs=-1)

rfr_cv.fit(train_input_scaled, train_target_scaled)
print(f"Best hyperparameters: {rfr_cv.best_params_}")
print(f"Best R² (train): {rfr_cv.best_score_}")

In [61]:
rfr_mod = RandomForestRegressor(max_depth=rfr_cv.best_params_["max_depth"], min_samples_split= rfr_cv.best_params_["min_samples_split"], n_estimators= rfr_cv.best_params_["n_estimators"])
                                
rfr_mod.fit(train_input_scaled, train_target_scaled)
# Predict on test set
val_prediction = rfr_mod.predict(val_input_scaled).reshape(-1, 1)
print(val_prediction.shape)
print(val_input_scaled.shape)
print(f"R²: {rsqr_score(val_target_scaled, val_prediction)}")

max_depth, learning rate, reg_lambda for xgBoost

In [62]:
max_depth = [int(x) for x in np.linspace(1, 10, num = 4)]
max_depth.append(None)
# no limit

learning_rate = [0.01, 0.05, 0.1]

n_estimators = [int(x) for x in np.linspace(start = 10, stop = 1000, num = 10)]

reg_lambda = [int(x) for x in np.linspace(start = 1, stop = 100, num = 10)]

param_list = {'max_depth': max_depth,
              'learning_rate': learning_rate,
              'reg_lambda': reg_lambda
             }
xgb_cv = RandomizedSearchCV(XGBRegressor(), param_distributions = param_list, 
                        n_iter=10, verbose =1, n_jobs=-1)

xgb_cv.fit(train_input_scaled, train_target_scaled)
print(f"Best hyperparameters: {xgb_cv.best_params_}")
print(f"Best R² (train): {xgb_cv.best_score_}")

In [71]:
xgb_mod = XGBRegressor(max_depth=xgb_cv.best_params_["max_depth"], learning_rate= xgb_cv.best_params_["learning_rate"], reg_lambda= xgb_cv.best_params_["reg_lambda"])
# Fit the model on train set
xgb_mod.fit(train_input_scaled, train_target_scaled)
# Predict on test set
val_prediction = xgb_mod.predict(val_input_scaled).reshape(-1, 1)
                                
print(val_prediction.shape)
print(val_input_scaled.shape)
print(f"R²: {rsqr_score(val_target_scaled, val_prediction)}")

In [77]:
max_depth = [int(x) for x in np.linspace(3, 12, num = 4)]
max_depth.append(None)
# no limit

learning_rate = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25]

lambda_l2 = [int(x) for x in np.linspace(start = 0, stop = 100, num = 10)]

param_list = {'max_depth': max_depth,
              'learning_rate': learning_rate,
              'lambda_l2': lambda_l2
             }
lgbm_cv = RandomizedSearchCV(LGBMRegressor(), param_distributions = param_list, 
                        n_iter=10, verbose =1, n_jobs=-1)

lgbm_cv.fit(train_input_scaled, train_target_scaled)
print(f"Best hyperparameters: {lgbm_cv.best_params_}")
print(f"Best R² (train): {lgbm_cv.best_score_}")

max_depth, learning_rate, lambda_l2 for LGBM 

In [78]:
lgbm_mod = LGBMRegressor(max_depth=lgbm_cv.best_params_["max_depth"], learning_rate= lgbm_cv.best_params_["learning_rate"], lambda_l2= lgbm_cv.best_params_["lambda_l2"])
# Fit the model on train set
lgbm_mod.fit(train_input_scaled, train_target_scaled)
# Predict on test set
val_prediction = lgbm_mod.predict(val_input_scaled).reshape(-1, 1)

print(val_prediction.shape)
print(val_input_scaled.shape)
print(f"R²: {rsqr_score(val_target_scaled, val_prediction)}")

# Submission

In [79]:
 scaled_test.shape

In [85]:
final_pred = np.exp(lgbm_mod.predict(scaled_test))

submission = pd.concat([Test_id,pd.Series(final_pred,name="SalePrice")],axis=1)
submission.to_csv("./submission.csv",index=False,header=True)
submission